In [1]:
import pandas as pd
import numpy as np
import gzip
import json
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
from transformers import pipeline

In [2]:
import json
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Appliances.jsonl.gz')

In [3]:
df = df.dropna(subset=['text'])

# Function to count words in a review
def count_words(text):
    return len(str(text).split())

# Filter reviews with more than 100 words
long_reviews = df[df['text'].apply(count_words) > 100].head(10)
long_reviews ['count'] = df['text'].apply(count_words)

In [4]:
summarizer = pipeline("summarization",model="sshleifer/distilbart-cnn-12-6")

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

C:\Users\Kirstin Ramos\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kirstin Ramos\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


In [5]:
print(summarizer.model.config._name_or_path)

sshleifer/distilbart-cnn-12-6


In [6]:
# Apply summarization
long_reviews['summary'] = long_reviews['text'].apply(lambda x: summarizer(x, max_length=80, min_length=50, do_sample=False)[0]['summary_text'])

# Display the first two summaries for your report
report_summaries = long_reviews[['text', 'summary']].head(2).reset_index()


In [7]:
report_summaries=report_summaries.reset_index(drop=True)


In [8]:
report_summaries['summary_count'] = report_summaries['summary'].apply(count_words)
report_summaries

,index,text,summary,summary_count
0,13,im excited to see how the coffee comes out i ...,i got 2 mason jars too so i ordered a 2nd on...,66
1,18,The filter did not fit in my Keurig K50. When ...,The filter did not fit in my Keurig K50 . Aft...,52


In [9]:
print("Original Review no.1")
print(report_summaries['text'][0])


print("Summary Review no.1")
print(report_summaries['summary'][0])


print('\n')
print('===========')
print("\n")


print("Original Review no.1")
print(report_summaries['text'][1])


print("Summary Review no.1")
print(report_summaries['summary'][1])

Original Review no.1
im excited to see how the coffee comes out  i got 2 mason jars too  so i ordered a 2nd one of these so i'll always have 2 in the fridge  for the summer<br /><br />UPDATE:  my first cup was a bit strong  i added way too much coffee i guess but i watered it down with plenty of ice and  half and half - a little cinnamon and vanilla -  i ordered the 2nd filter and it will be here soon and i plan to have iced coffee all summer ...  definitely cutting down on the amount of coffee though !!!!!
Summary Review no.1
 i got 2 mason jars too  so i ordered a 2nd one of these so i'll always have 2 in the fridge . i added way too much coffee i guess but i watered it down with plenty of ice and  half and half . i ordered the 2nd filter and it will be here soon and i plan to have iced coffee all summer .




Original Review no.1
The filter did not fit in my Keurig K50. When closing the keurig, there was still a small gap that left the keurig slightly open. I had to hold down the ke

In [11]:
#df['text'][62]

text = '''I have used these in my Samsung fridge for almost 2 years now and the water quality is better than the other products that fit in the fridge and there is no difference with the Samsung filter, maybe even slightly better.'''


reader = pipeline("question-answering",model="distilbert/distilbert-base-cased-distilled-squad")
question = "What brand of fridge he is using?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

Device set to use cpu


,score,start,end,answer
0,0.705326,24,31,Samsung
